In [2]:
%pip install chromadb
%pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached sentence_transformers-3.3.1-py3-none-any.whl (268 kB)
Using cached torch-2.5.1-cp312-cp312-win_amd64.whl (203.0 MB)
Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [25]:
token_huggingface ="Introduce tu token" 

In [10]:
import chromadb
client = chromadb.PersistentClient(path="./data")

In [44]:
from sentence_transformers import SentenceTransformer
 ## Funcion para obtener vector de embeddings
def obtener_vector_embeddings(texto_documento: str) -> list:
    """
    Función para generar un vector de embeddings a partir del texto utilizando el modelo Sentence-BERT.

    Args:
    texto_documento (str): El texto del cual se desea generar el embedding.

    Returns:
    list: Un vector de embeddings generado a partir del texto.
    """
    # Cargar el modelo de embeddings (Sentence-BERT)
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2',use_auth_token=token_huggingface)  # Modelo ligero y rápido

    # Generar el vector de embeddings a partir del texto
    vector_documento = model.encode(texto_documento).tolist()  # Convierte el vector a lista

    return vector_documento


In [ ]:
# Crear una colección
collection = client.get_or_create_collection(
    name="space_x_technology")


In [15]:

#  documento sobre SpaceX
texto_documento = """
# Alcances de la Tecnología de SpaceX

SpaceX es una empresa aeroespacial privada fundada por Elon Musk con el objetivo de reducir los costos de los viajes espaciales y hacer posible la colonización de Marte. La compañía ha logrado avances tecnológicos significativos que han cambiado la dinámica de la industria espacial.

## Tecnología Clave

### Cohetes Reutilizables
Uno de los avances más importantes de SpaceX ha sido el desarrollo de cohetes reutilizables. El **Falcon 9** y su variante más grande **Falcon Heavy** son capaces de aterrizar después de lanzar su carga al espacio, lo que reduce drásticamente los costos de los viajes espaciales.

### Starship
El **Starship** es la nave espacial diseñada para viajes interplanetarios, capaz de transportar humanos y carga a la Luna, Marte y más allá. Utiliza el sistema de propulsión **Raptor**, que promete un rendimiento superior a los motores tradicionales.

### Starlink
Otro proyecto clave de SpaceX es **Starlink**, una constelación de satélites en órbita baja terrestre que proporciona acceso a internet de alta velocidad y baja latencia a nivel global, especialmente en áreas rurales o de difícil acceso.

## Logros y Misiones

SpaceX ha logrado varios hitos en la industria aeroespacial, incluyendo:

- **Transporte de carga a la ISS**: SpaceX ha sido responsable de varias misiones exitosas de transporte de carga a la Estación Espacial Internacional.
- **Misiones Tripuladas**: En 2020, SpaceX lanzó con éxito la primera misión tripulada a la ISS, marcando la primera vez que una empresa privada enviaba astronautas al espacio.
- **Colonización de Marte**: A largo plazo, el objetivo de SpaceX es desarrollar una tecnología que permita la colonización de Marte, con la Starship como la pieza central de esta visión.

## Impacto en la Industria Aeroespacial

SpaceX ha cambiado la industria de la aviación y los viajes espaciales mediante la reducción de costos y la innovación tecnológica. La reutilización de cohetes y la producción en masa de satélites han hecho los viajes espaciales más accesibles y económicos. Además, SpaceX ha sido un competidor clave para agencias espaciales tradicionales como la NASA, proporcionando un enfoque comercial y eficiente.

## Perspectivas Futuras

SpaceX continúa desarrollando nuevas tecnologías con la visión de establecer una presencia humana en Marte y más allá. Los proyectos para viajar a la Luna, en colaboración con NASA, y la expansión de **Starlink** hacia mercados internacionales son algunas de las iniciativas futuras.

"""


In [18]:
class MetadatosDocumento:
    def __init__(self, categoria, autor, fecha_creacion, tema, lenguaje):
        self.categoria = categoria
        self.autor = autor
        self.fecha_creacion = fecha_creacion
        self.tema = tema
        self.lenguaje = lenguaje
    
    # Método para convertir el objeto a diccionario
    def to_dict(self):
        return {
            "categoria": self.categoria,
            "autor": self.autor,
            "fecha_creacion": self.fecha_creacion,
            "tema": self.tema,
            "lenguaje": self.lenguaje
        }


In [40]:
## CREAR

def crear_documento(collection, texto_documento: str, document_id: str, metadatos_objeto) -> dict:
    """
    Función para crear un documento en la colección de Chroma DB, pasando un objeto como metadatos.
    
    Args:
    collection (chromadb.Collection): La colección donde se agregará el documento.
    texto_documento (str): El texto del documento.
    document_id (str): El ID único del documento.
    metadatos_objeto (object): El objeto que contiene los metadatos, debe tener el método to_dict().
    
    Returns:
    dict: Documento creado en la colección.
    """
    
    # Validación: Asegurarse de que el texto no esté vacío
    if not texto_documento.strip():
        raise ValueError("El texto del documento está vacío. No se puede crear el documento.")
    
    # Validación: Asegurarse de que el ID no esté duplicado 
    existing_document = collection.get(ids=[document_id])
    if existing_document["documents"]:
        raise ValueError(f"Ya existe un documento con ID '{document_id}'. El ID debe ser único.")
    
    # Convertir el objeto a diccionario
    metadatos_documento = metadatos_objeto.to_dict()
    
    # Obtener el embedding del documento
    embeddings = obtener_vector_embeddings(texto_documento)

    # Crear el documento en la colección
    document = collection.add(
        ids=[document_id],  # Identificador único del documento
        documents=[texto_documento],  # El texto del documento
        metadatas=[metadatos_documento],  # Metadatos asociados al documento
        embeddings=[embeddings],  # El vector real generado por el modelo
    )

    print(f"Documento con ID '{document_id}' creado exitosamente.")
    
    return document

In [33]:
## LEER
def leer_documento(collection, document_id: str):
    """
    Función para leer un documento de la colección de Chroma DB utilizando su ID.

    Args:
    collection: La colección de Chroma DB donde está almacenado el documento.
    document_id (str): El ID del documento que se desea recuperar.

    Returns:
    None
    """
    # Recuperar el documento por ID
    document = collection.get(ids=[document_id])

    # Verificar si se encontró el documento
    if document["documents"]:
        print(f"Documento encontrado: {document['documents'][0]}\n")
        print(f"Metadatos: {document['metadatas'][0]}\n")
        
        # Mostrar los embeddings (si están disponibles)
        if document["embeddings"]:
            print(f"Embeddings (primer valor): {document['embeddings'][0]}")
        else:
            print("No se encontraron embeddings para este documento.")
    else:
        print(f"No se encontró un documento con ID '{document_id}'.")


In [31]:
## ACTUALIZAR
from datetime import datetime

def actualizar_documento(collection, document_id: str, texto_nuevo: str):
    """
    Función para actualizar un documento en la colección de Chroma DB.

    Args:
    collection: La colección de Chroma DB donde se encuentra el documento.
    document_id (str): El ID del documento a actualizar.
    texto_nuevo (str): El nuevo texto que reemplazará al documento actual.

    Returns:
    None
    """
    # Paso 1: Verificar si el documento con el ID existe
    document = collection.get(ids=[document_id])

    if not document["documents"]:
        print(f"No se encontró un documento con ID '{document_id}'. No se puede actualizar.")
        return

    # Validaciones adicionales antes de la actualización
    # Generar un embedding del nuevo texto
    vector_nuevo = obtener_vector_embeddings(texto_nuevo)  # Generar el embedding

    # Validar que el embedding se haya generado correctamente
    if not vector_nuevo or len(vector_nuevo) == 0:
        print("No se pudo generar el embedding para el nuevo texto.")
        return

    # Paso 2: Eliminar el documento anterior
    print(f"Eliminando documento con ID '{document_id}'...")
    collection.delete(ids=[document_id])
    print("Documento eliminado exitosamente.")

    # Paso 3: Crear los nuevos metadatos
    metadatos_nuevos = {
        "categoria": "tecnología espacial",
        "autor": "Equipo de Investigación",
        "fecha_creacion": str(datetime.now()),  # Usar datetime.datetime.now() si es necesario
        "tema": "SpaceX y la exploración espacial",
        "lenguaje": "español"
    }

    # Paso 4: Agregar el documento actualizado a la colección
    print(f"Agregando el documento actualizado con ID '{document_id}'...")
    document = collection.add(
        ids=[document_id],  # Usamos el mismo ID para mantenerlo
        documents=[texto_nuevo],
        metadatas=[metadatos_nuevos],
        embeddings=[vector_nuevo]
    )

    print("Documento actualizado exitosamente.")


In [35]:
def eliminar_documento(collection, document_id: str):
    """
    Función para eliminar un documento de la colección de Chroma DB utilizando su ID.

    Args:
    collection: La colección de Chroma DB donde está almacenado el documento.
    document_id (str): El ID del documento que se desea eliminar.

    Returns:
    None
    """
    # Paso 1: Verificar si el documento existe antes de eliminarlo
    document = collection.get(ids=[document_id])

    if document["documents"]:
        # Paso 2: Eliminar el documento
        collection.delete(ids=[document_id])
        print(f"Documento con ID '{document_id}' eliminado exitosamente.")
    else:
        print(f"No se encontró un documento con ID '{document_id}'. No se puede eliminar.")


In [ ]:
## CRUD
# Crear una conexión al cliente Chroma
# Consumir funcion CREAR
client = chromadb.Client()
import datetime

# Seleccionar o crear una colección (si no existe ya)
collection = client.get_collection("space_x_technology")

# Crear una instancia de los metadatos como objeto
metadatos_documento = MetadatosDocumento(
    categoria="tecnología espacial",
    autor="Equipo de Investigación",
    fecha_creacion=str( datetime.datetime.now()),
    tema="SpaceX y la exploración espacial",
    lenguaje="español"
)

# Texto de ejemplo para el documento
texto_documento = texto_documento # puede ir cualquier texto

# ID único para el documento
document_id = "space_x_document"

# Llamar a la función para crear el documento
document = crear_documento(collection, texto_documento, document_id, metadatos_documento)


c:\Users\namado\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [38]:
# Consumir funcion LEER
collection = client.get_collection("space_x_technology")

# ID del documento que deseas leer
document_id = "space_x_document"

# Llamada a la función de lectura
leer_documento(collection, document_id)


Documento encontrado: 
# Alcances de la Tecnología de SpaceX

SpaceX es una empresa aeroespacial privada fundada por Elon Musk con el objetivo de reducir los costos de los viajes espaciales y hacer posible la colonización de Marte. La compañía ha logrado avances tecnológicos significativos que han cambiado la dinámica de la industria espacial.

## Tecnología Clave

### Cohetes Reutilizables
Uno de los avances más importantes de SpaceX ha sido el desarrollo de cohetes reutilizables. El **Falcon 9** y su variante más grande **Falcon Heavy** son capaces de aterrizar después de lanzar su carga al espacio, lo que reduce drásticamente los costos de los viajes espaciales.

### Starship
El **Starship** es la nave espacial diseñada para viajes interplanetarios, capaz de transportar humanos y carga a la Luna, Marte y más allá. Utiliza el sistema de propulsión **Raptor**, que promete un rendimiento superior a los motores tradicionales.

### Starlink
Otro proyecto clave de SpaceX es **Starlink**, 

In [ ]:
# Consumir funcion Actualizar
collection = client.get_collection("space_x_technology")

# ID del documento que deseas actualizar
document_id = "space_x_document"

# Nuevo texto que reemplazará el texto actual
texto_nuevo = "SpaceX está llevando a cabo una investigación avanzada para la colonización de Marte y la expansión del comercio espacial."

# Llamada a la función de actualización
actualizar_documento(collection, document_id, texto_nuevo)


Eliminando documento con ID 'space_x_document'...
Documento eliminado exitosamente.
Agregando el documento actualizado con ID 'space_x_document'...
Documento actualizado exitosamente.
Documento actualizado: None


In [54]:
# Consumir funcion ELIMINAR
# Supongamos que ya tienes el cliente y la colección seleccionados
collection = client.get_collection("space_x_technology")

# ID del documento que deseas eliminar
document_id = "space_x_document"

# Llamada a la función para eliminar el documento
eliminar_documento(collection, document_id)


Documento con ID 'space_x_document' eliminado exitosamente.
